<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/telebot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install telebot
import telebot
import sqlite3
from datetime import datetime

# Создаем объект бота и указываем токен
with open('token.txt') as token:
    key = token.readline()

bot = telebot.TeleBot(key)

# Создаем контекстный менеджер для работы с базой данных
class Database:
    def __init__(self):
        self._conn = sqlite3.connect('messages.db')
        self._cursor = self._conn.cursor()

        # Создаем таблицу для хранения сообщений
        self._cursor.execute('''CREATE TABLE IF NOT EXISTS messages
                                (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                 sender TEXT,
                                 message TEXT,
                                 date TEXT)''')
        self._conn.commit()

    def __enter__(self):
        return self._cursor

    def __exit__(self, exc_type, exc_val, exc_tb):
        self._conn.commit()
        self._conn.close()

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start_handler(message):
    bot.reply_to(message, 'Привет! Я бот для хранения сообщений.')

# Обработчик всех остальных сообщений
@bot.message_handler(func=lambda message: True)
def message_handler(message):
    # Получаем информацию об отправителе и времени получения сообщения
    sender = message.from_user.username
    date = datetime.fromtimestamp(message.date).strftime('%Y-%m-%d %H:%M:%S')

    # Сохраняем сообщение в базу данных
    with Database() as cursor:
        cursor.execute("INSERT INTO messages (sender, message, date) VALUES (?, ?, ?)",
                       (sender, message.text, date))

    bot.reply_to(message, 'Сообщение сохранено!')

# Запускаем бота
bot.polling()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
